In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("split_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from utils.config import Config
from utils.parse_dataset import NetworkDataset, load_datasets
from utils.load_models import LSTM_Models

In [2]:
conf = Config()
load_models = LSTM_Models()
model = load_models.split_lstm_4

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513, 1]) torch.Size([512])


In [5]:
acc, train_loss, val_loss = model.train(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 84.03%, Train loss: 0.8521, Val loss: 0.2851
Epoch: 2/10, Accuracy: 91.59%, Train loss: 0.2421, Val loss: 0.1447
Epoch: 3/10, Accuracy: 93.49%, Train loss: 0.1497, Val loss: 0.1031
Epoch: 4/10, Accuracy: 94.56%, Train loss: 0.1142, Val loss: 0.0823
Epoch: 5/10, Accuracy: 95.23%, Train loss: 0.0968, Val loss: 0.0727
Epoch: 6/10, Accuracy: 95.42%, Train loss: 0.0868, Val loss: 0.0670
Epoch: 7/10, Accuracy: 95.59%, Train loss: 0.0796, Val loss: 0.0619
Epoch: 8/10, Accuracy: 95.83%, Train loss: 0.0745, Val loss: 0.0586
Epoch: 9/10, Accuracy: 96.18%, Train loss: 0.0706, Val loss: 0.0559
Epoch: 10/10, Accuracy: 96.33%, Train loss: 0.0675, Val loss: 0.0538


In [6]:
loss, accuracy = model.evaluate(test_loader)
print(f" Accuracy: {100*accuracy:.2f}%")

 Accuracy: 96.25%


In [7]:
model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/split_models/checkpoint/split_lstm_4.pth
